### <center> Nouveau scoring des entités (et sauvegarde en .csv)

#### Importation des modules, chargement des données et des modèles de nlp

In [11]:
import spacy
import pandas as pd
import numpy as np
import neuralcoref
import ast
import random
from tqdm import tqdm_notebook

data_raw = pd.read_csv('photos.csv',encoding='latin-1')
scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}

for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]

nlp=spacy.load("en_core_web_md")
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

C:\Users\THINKPAD\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,15,17,23,24,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Réduction de la base initiale

In [12]:
print(len(data_raw))
data_en=data_raw[data_raw['language']=='english']
print(len(data_en))
data1=data_en.dropna(subset=['event'])
print(len(data1))
data2=data1.drop_duplicates(subset='caption', keep='first')
print(len(data2))
df=data2[[(len(descr.split())>10) for descr in data2.caption]]
len(df)


360999
360723
245475
154032


154030

#### Définition de la 2nde base de travail basée sur le lien avec les dépêches

On regarde tout d'abord le nombre d'évènements uniques dans la base des dépêches et dans notre base de photos, puis on regarde combien d'évènements sont en commun.

In [13]:
depeches=pd.read_csv('depeches.csv')
liste_ev_unique_depeches = []
for ev in depeches.event:
    for _ev in ev[1:-1].replace("'","").split(', '):
        if _ev not in liste_ev_unique_depeches:
            liste_ev_unique_depeches.append(_ev)
print(len(liste_ev_unique_depeches))

liste_ev_unique_photos = []
for ev in df.event:
    if(type(ev) != float):
        for _ev in ev[1:-1].replace("'","").split(', '):
            if _ev not in liste_ev_unique_photos:
                liste_ev_unique_photos.append(_ev)
print(len(liste_ev_unique_photos))

ev_in = []
for ev in liste_ev_unique_photos:
    if(ev in liste_ev_unique_depeches):
        ev_in.append(ev)
print(len(ev_in), 'en commun')

13335
16055
2979 en commun


Pour définir la 2nde base de travail, on doit filtrer la 1ère base pour ne conserver que les photos associées à au moins un évènement de la base de dépêches et réciproquement avec la base de dépêches. Mais avant, de faire ce filtre, il faut opérer une transformation sur les évènements pour pouvoir les exploiter.

In [14]:
def transfo_event(ev):
    res=ev
    return res[1:-1].replace("'","").split(', ')

def ev_in_depeche(ev_list):
    for ev in ev_list:
        if ev in liste_ev_unique_depeches:
            return True
    return False

def ev_in_photos(ev_list):
    for ev in ev_list:
        if ev in liste_ev_unique_photos:
            return True
    return False

df_c=df.copy()
df_c.event=df_c.event.apply(transfo_event)
df2=df_c.copy()
df2=df2[[ev_in_depeche(ev_list) for ev_list in tqdm_notebook(df_c.event)]]
len(df2)

depeche_c=depeches.copy()
depeche_c.event=depeche_c.event.apply(transfo_event)
df_depeche=depeche_c.copy()
df_depeche=df_depeche[[ev_in_photos(ev_list) for ev_list in tqdm_notebook(depeche_c.event)]]

On va dans un 1er temps travailler uniquement sur df2 car elle est moins volumineuse et permet d'utiliser les dépêches pour tester l'efficacité de l'algorithme de similarité (défini dans Tests de similarité).

#### Algorithme de scoring des entités
On définit d'abord quelques fonctions intermédiaires.

In [15]:
def dep_ent(ent, doc):
    """ Retourne la fonction grammaticale :  la 'dep', d'une entité. Cette fonction est nécessaire car elle permet d'affecter
    une dep à une entité composée de plusieurs mots ayant chacun une dep de base.
    Traite aussi le cas particulier des mots étant des conj ou des compound : leur vrai dep et celle du mot auxquels
    ils sont associés en tant que conj ou compound."""
    start= ent.start
    end=ent.end
    for k in range(start,end):
        if doc[k].head.text not in ent.text: 
            if doc[k].dep_=='conj':     
                tok=doc[k]            
                while tok.dep_=='conj':
                    tok=tok.head      
                return(tok.dep_)
            
            if doc[k].dep_=='compound':   
                tok=doc[k]            
                while tok.dep_=='compound':
                    tok=tok.head      
                return(tok.dep_)
            return(doc[k].dep_)    
    return doc[start].dep_

def ent_good_type(ent): #filtre les entités selon leur type
    return (ent.label_ == "PERSON" or ent.label_ == "NORP" or ent.label_ == "ORG" or ent.label_ == "GPE" or ent.label_ == "EVENT" or ent.label_ == "LOC")

def get_resolved(doc, cluster):
    resolved = list(tok.text_with_ws for tok in doc)
    for coref in cluster:
        if coref.text.lower() != cluster.main.text.lower():
            resolved[coref.start] = cluster.main.text + doc[coref.end-1].whitespace_
            for i in range(coref.start+1, coref.end):
                resolved[i] = ""
    return ''.join(resolved)

def add_to_dic(dic,ent,doc):
    if ent.text not in dic.keys():
        dic[ent.text]=dict_val[dep_ent(ent,doc)]
    else:
        dic[ent.text]+=dict_val[dep_ent(ent,doc)]

Et maintenant l'algorithme de scoring en lui même:

In [16]:
def scores_doc_coref4(doc):
    ent_list=[ent for ent in doc.ents if ent_good_type(ent)]
    clusters=doc._.coref_clusters
    res={}
    for ent in ent_list:
        if ent._.is_coref:
            for cluster in clusters:
                if ent in cluster.mentions:
                    doc_res=nlp(get_resolved(doc,cluster))
                    for ent1 in doc_res.ents:
                        if ent1.text==ent.text:
                            add_to_dic(res,ent1,doc_res)
        else:
            flag=0
            clust_count=0
            for cluster in clusters:
                ent_in_cluster=False
                for span in cluster.mentions:
                    if ent.text in span.text:
                        ent_in_cluster=True
                        clust_count+=1
                        break
                if ent_in_cluster and ent.label_ != 'NORP':
                    flag=1
                    doc_res=nlp(get_resolved(doc,cluster))
                    for ent1 in doc_res.ents:
                        if ent1.text==ent.text:
                            add_to_dic(res,ent1,doc_res)    
            if flag==0: 
                add_to_dic(res,ent,doc)
    return res

#### Exécution du nlp sur chaque caption de df2 et df_depeche

In [157]:
nlp_list=[]
for desc in tqdm_notebook(df2.caption):
    nlp_list.append(nlp(desc))

#### Calcul du score de chaque caption

In [162]:
score_list = [scores_doc_coref4(doc) for doc in tqdm_notebook(nlp_list)]

#### Création du csv
On convertit les dictionnaires en str pour pouvoir ensuite les stocker dans un csv.

In [163]:
score_list_str=[str(dic) for dic in tqdm_notebook(score_list)]

In [166]:
df_scores=pd.DataFrame({'description':df2.caption,'scores':score_list_str})
df_scores.to_csv("scores4.csv",index=False)

### <center> Nouveaux tests de similarité


#### Lecture du csv

In [17]:
df_read=pd.read_csv("scores4.csv")
liste_scores4=[]
for string in df_read.scores:
    liste_scores4.append(ast.literal_eval(string)) 
liste_scores4[0]

{'Portugal': 0.38297872299999997,
 'Antonio Costa': 0.46808510600000003,
 'the European Council': 0.8297872340000001,
 'Charles Michel': 1.0,
 'European Council': 0.8297872340000001,
 'Brussels': 0.8297872340000001,
 'the European Union': 0.8297872340000001,
 'EU': 0.46808510600000003,
 'Marin': 0.0}

#### Score de similarité
On commence par définir un score de similarité entre un doc (article ou autre) et la caption d'une image ainsi qu'une fonction intermédiaire utile pour réduire le nombre de comparaison dans l'algorithme final.

In [24]:
def score_sim4(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref1 : on utilise liste_scores1.
    Par rapport aux fonctions précédentes, ici on a changé le calcul du score de similarité en "normalisant" les scores
    du doc et de l'image, au lieu de les diviser par 2 (ce qui était inutile)"""
    res=0
    score_image=liste_scores4[k]
    max_doc=max(score_doc.values())
    max_im=max(score_image.values())
    for i in score_doc.keys():
        if i in score_image.keys():
            if max_doc==0 or max_im==0:
                return 0
            res += score_doc[i]/max_doc +score_image[i]/max_im
    return res

def related_descr(doc):
    """Retourne la liste des indices correspondant aux descriptions ayant au moins 1 entité en commun avec le doc.
    Permet d'effectuer moins de comparaison dans la fonction best_image."""
    index_list=[]
    for k in range(len(liste_scores4)):
        for ent in doc.ents:
            if ent.text in liste_scores4[k].keys():
                index_list.append(k)
                break
    return index_list

On définit maintenant l'algorithme final qui doit retourner l'indice de l'image correspondant le mieux au texte selon le score de similarité précédent.
On définit aussi un algorithme qui retourne les indices (et scores) des 5 images correspondant le mieux au texte selon le score de similarité précédent.

In [25]:
def best_image4(doc):
    """Retourne l'indice de l'image dont la description correspond le plus au doc passé en paramètre.
    La correspondance (matching) se fait avec score_sim3."""
    score_doc=scores_doc_coref4(doc)
    best_score=0
    best_descr=0
    for k in related_descr(doc):
        if score_sim4(doc,score_doc,k) > best_score:
            best_score=score_sim4(doc,score_doc,k)
            best_descr=k
    return best_descr

def best5_image4(doc):
    rel_descr=related_descr(doc)
    score_doc=scores_doc_coref4(doc)
    best_scores=[(i,score_sim4(doc,score_doc,i)) for i in rel_descr[:5]]
    best_scores.sort(key = lambda x : x[1])
    for k in rel_descr[5:]:
        score_sim=score_sim4(doc,score_doc,k)
        if score_sim > best_scores[0][1]:
            best_scores.pop(0)
            best_scores.append((k,score_sim4(doc,score_doc,k)))
            best_scores.sort(key = lambda x : x[1])
    return best_scores

#### Tests


df2 df_read df_depeche
Ce qu'on doit faire : prendre une dépêche au hasard. Calculer nlp sur sa description, calculer score.
Faire une fonction 5_best qui renvoie les 5 meilleures descriptions. On peut faire un test et si l'event de la depeche est dedans on compte juste.

In [27]:
df2.reset_index(drop=True,inplace=True)
df_depeche.reset_index(drop=True,inplace=True)

In [74]:
def ev_match(ev_list1,ev_list2):
    for ev in ev_list1:
        if ev in ev_list2:
            return True
    return False

def test(i):
    print('Depeche n°',i,'(events = ',df_depeche.event[i],')\n',df_depeche.news2[i],'\n\n Best 5 images :\n')
    for tup in best5_image4(nlp(df_depeche.news2[i])):
        print('photo n°',tup[0],'(events = ',df2.event[tup[0]],')\n',df_read.description[tup[0]],'\n')
    
def perf_score4():
    

#a=best_image4(nlp(df_depeche.news2[2]))
#print('\n',a)
#print(df_read.description[a],df2.event[a])

def photo_filter_by_event(event_list):
    df3=df2.copy()
    df3=df2[[ev_match(ev_list,event_list)  for ev_list in df2.event]]
    df3.reset_index(drop=True,inplace=True)
    pd.set_option('display.max_colwidth',-1)
    display(df3[['caption','event']])
    pd.reset_option('display.max_colwidth')


In [50]:
test(2) #ça marche

Depeche n° 2 (events =  ['afpevent:FNR78', 'afpevent:FCS97', 'afpevent:FCQ20'] )
 One of two candidates in Guinea-Bissau's presidential elections said he would be sworn in on Thursday, even though a bitter row over the vote remains unresolved after nearly two months.
 Umaro Sissoco Embalo made the announcement on social media on Wednesday, referring to a decision in his favour by the country's electoral commission.
 Embalo, a 47-year-old former general and prime minister, won 53.55 percent of the votes in the December 29 elections, according to the National Electoral Commission.
 The Supreme Court, responding to a petition filed by the PAIGC, has issued rulings requiring a check of the vote tally sheets.
 On Tuesday, the election commission confirmed the results that it had announced, while the PAIGC stood by its objections.
 Guinea-Bissau, a West African state that nurses a reputation for graft and as a transit point for cocaine smuggling, has had a long history of coups and instabili

In [51]:
test(3) #marche pas car l'évènement est trop vague

Depeche n° 3 (events =  ['afpevent:FFQ37', 'afpevent:EDU79', 'afpevent:EDZ99'] )
 Inter Milan's Europa League last-32 second-leg tie against Ludogorets at the San Siro will be played without fans due to the coronavirus outbreak in Italy, UEFA announced on Wednesday.
 Inter are set to host the Bulgarian outfit on Thursday with the Italian side leading 2-0 after the second leg. 

 Best 5 images :

photo n° 7840 (events =  ['afpevent:ELD62', 'afpevent:EKX22', 'afpevent:ELD58'] )
 Inter Milan's Italian head coach Antonio Conte shouts shouts instructions to his players from the touchline during the Italian Serie A football match Inter Milan vs Cagliari on January 26, 2020 at the San Siro stadium in Milan. / AFP / Miguel MEDINA 

photo n° 7848 (events =  ['afpevent:ELD62', 'afpevent:EKX22', 'afpevent:ELD58'] )
 Inter Milan's Italian head coach Antonio Conte reacts as the ball flies across the touchline during the Italian Serie A football match Inter Milan vs Cagliari on January 26, 2020 at t

In [75]:
photo_filter_by_event(['afpevent:EDU79'])

,caption,event
0,"(FILES) In this file photo taken on December 12, 2019 Espanyol's Spanish coach Pablo Machin looks on before the UEFA Europa League Group H football match between Espanyol and CSKA Moscow at the RCDE Stadium in Cornella de Llobregat near Barcelona, on December 12, 2019. Espanyol, who are languishing at the bottom of La Liga, on December 23 fired chief coach Pablo Machin after just over two months in the job, the club said. / AFP / Josep LAGO","[afpevent:EDZ72, afpevent:EDU79, afpevent:EDZ57]"
1,"(From L) UEFA deputy general secretary Giorgio Marchetti, UEFA Europa League ambassador Josephine Henning, UEFA Europa League ambassador Frederic Kanoute, and UEFA head of club competitions Michael Heselschwerdt attend the UEFA Europa League football cup round of 32 draw ceremony on December 16, 2019 in Nyon. / AFP / Fabrice COFFRINI","[afpevent:FBR56, afpevent:EDZ82, afpevent:EDU79]"
2,"An official proceeds to the UEFA Europa League football cup round of 32 draw ceremony on December 16, 2019 in Nyon. / AFP / Fabrice COFFRINI","[afpevent:FBR56, afpevent:EDZ82, afpevent:EDU79]"
3,"UEFA Europa League ambassador Frederic Kanoute proceeds to the UEFA Europa League football cup round of 32 draw ceremony on December 16, 2019 in Nyon. / AFP / Fabrice COFFRINI","[afpevent:FBR56, afpevent:EDZ82, afpevent:EDU79]"
4,"UEFA Europa League ambassador Josephine Henning proceeds to the UEFA Europa League football cup round of 32 draw ceremony on December 16, 2019 in Nyon. / AFP / Fabrice COFFRINI","[afpevent:FBR56, afpevent:EDZ82, afpevent:EDU79]"
5,"An UEFA official removes the UEFA Europa League football cup trophy after the cup's round of 32 draw ceremony on December 16, 2019 in Nyon. / AFP / Fabrice COFFRINI","[afpevent:FBR56, afpevent:EDZ82, afpevent:EDU79]"
6,"The UEFA Europa League football cup trophy is pictured before the cup's round of 32 draw ceremony on December 16, 2019 in Nyon. / AFP / Fabrice COFFRINI","[afpevent:FBR56, afpevent:EDZ82, afpevent:EDU79]"
7,"UEFA Europa League ambassador Josephine Henning holds the slip of AS Roma during the UEFA Europa League football cup round of 32 draw ceremony on December 16, 2019 in Nyon. / AFP / Fabrice COFFRINI","[afpevent:FBR56, afpevent:EDZ82, afpevent:EDU79]"
8,"UEFA Europa League ambassador Josephine Henning holds the slip of FC Shakhtar Donetsk during the UEFA Europa League football cup round of 32 draw ceremony on December 16, 2019 in Nyon. / AFP / Fabrice COFFRINI","[afpevent:FBR56, afpevent:EDZ82, afpevent:EDU79]"
9,"UEFA Europa League ambassador Josephine Henning holds the slip of Rangers FC during the UEFA Europa League football cup round of 32 draw ceremony on December 16, 2019 in Nyon. / AFP / Fabrice COFFRINI","[afpevent:FBR56, afpevent:EDZ82, afpevent:EDU79]"


In [52]:
test(0) #marche pas (faudrait que corona soit detecté)

Depeche n° 0 (events =  ['afpevent:FHL21'] )
 Iranian cyberpolice have arrested 24 people accused of online rumour-mongering about the spread of the coronavirus in the country, semi-official news agency ISNA reported on Wednesday.
 The arrests were carried out after the establishment of a special unit to "combat rumour-mongers regarding the 'spread of coronavirus in the country'," he was quoted as saying. 

 Best 5 images :

photo n° 13440 (events =  ['afpevent:FHQ48'] )
 EDITORS NOTE: Graphic content / TOPSHOT - Rescue teams recover debris from a field after a Ukrainian plane carrying 176 passengers crashed near Imam Khomeini airport in the Iranian capital Tehran early in the morning on January 8, 2020, killing everyone on board. The Boeing 737 had left Tehran's international airport bound for Kiev, semi-official news agency ISNA said, adding that 10 ambulances were sent to the crash site. / AFP / - 

photo n° 13441 (events =  ['afpevent:FHQ24'] )
 TOPSHOT - People and rescue teams ar

In [54]:
test(1) #marche

Depeche n° 1 (events =  ['afpevent:FHL21'] )
 There are now more new cases of the coronavirus reported each day outside China than inside the hardest-hit country, the World Health Organization said on Wednesday.
 While new case numbers and deaths are dwindling at the disease epicentre in China, the country remains by far the hardest hit. 

 Best 5 images :

photo n° 9587 (events =  ['afpevent:FHL21'] )
 A passenger wears a face mask on board of a plane at Yangon International Airport on January 21, 2020. China has confirmed human-to-human transmission in the outbreak of a new SARS-like virus as the number of cases soared and authorities January 21 said a fourth person had died, as the World Health Organization said it would consider declaring an international public health emergency over the outbreak.
 / AFP / Mladen ANTONOV 

photo n° 9588 (events =  ['afpevent:FHL21'] )
 Passengers wearing face masks board their flight at Yangon International Airport on January 21, 2020. China has co

In [67]:
test(5) #bof : faudrait détecter animaux

Depeche n° 5 (events =  ['afpevent:FHL21'] )
 The novel coronavirus outbreak in China may end up saving one of the world's most trafficked animals after Beijing announced a total ban on the sale and consumption of the pangolin.
 Beijing implemented similar measures following the SARS outbreak in the early 2000s, but the trade and consumption of wild animals, including bats and snakes, made a comeback.  

 Best 5 images :

photo n° 6508 (events =  ['afpevent:FHL21'] )
 TOPSHOT - People wear protective masks as they walk outside a shopping mall in Beijing on January 23, 2020. China is halting public transport and closing highway toll stations in two more cities in Hubei province, the epicentre of a deadly virus outbreak, authorities said on January 23. / AFP / NICOLAS ASFOURI 

photo n° 6522 (events =  ['afpevent:FHL21'] )
 TOPSHOT - Security personnel wearing protective masks walk in front of the portrait of late communist leader Mao Zedong (C, back) at Tiananmen Gate in Beijing on Janu

In [73]:
#test(100)

In [72]:
#photo_filter_by_event(['afpevent:FNN22'])

,caption,event


In [ ]:
nlp_depeche=[]
for desc in tqdm_notebook(df_depeche.news2):
    doc=nlp(desc)
    if len(doc.ents)>50:
        last_ent=doc.ents[50]
        
        doc=nlp(doc[:last_ent.end].text)
    nlp_depeche.append(doc)

len_ent=pd.Series([len(doc.ents) for doc in nlp_depeche])
len_ent.quantile(0.875)

On commence par créer une liste d'indices à 100 images au hasard dans la base, puis on calcule le pourcentage de caption i dont best_image retourne i (i.e l'image la plus proche de la caption est l'image correspondant à cette caption).

In [79]:
rand_ind_list=[]
for i in range(10):
    num = random.randint(0, df_depeche.shape[0])
    while(num in rand_ind_list):
        num = random.randint(0, df_depeche.shape[0])
    rand_ind_list.append(num)
print(rand_ind_list)

count=0
count2=0
#rand_int_list=range(100)
for i in tqdm_notebook(rand_ind_list):
    best5=best5_image4(nlp(df_depeche.news2[i]))
    flag=0
    tup_ind=0
    for tup in best5:
        if ev_match(df2.event[tup[0]],df_depeche.event[i]):
            flag=1
            if tup_ind==4:
                count+=1
        tup_ind+=1
    count2+=flag
    print(count,count2)
print(count*10,'% \n', count2*10,'%')

[8946, 6982, 1481, 2147, 5512, 5324, 6439, 8804, 9710, 7428]


0 1
1 2
2 3
3 4
3 4
3 4
3 5
4 6
4 6
5 7
50 % 
 70 %


In [80]:
test(9710)

Depeche n° 9710 (events =  ['afpevent:FAO84'] )
 Landlocked Bolivia, in crisis after its president quit amid protests over a disputed election, is among Latin America's poorest countries despite having huge gas reserves. It is home to the region's largest indigenous population.
 Here are some key facts about the country of 11 million people:
 Evo Morales became the first indigenous president of Bolivia in 2006 after a landmark election victory broke decades of domination by an elite largely of European or mixed descent.
 Around 36 indigenous languages are officially recognised -- including Quechua, Aymara and Guarani -- along with an indigenous legal system different to state law.
 Morales had been the region's longest-serving leader, credited with bringing relative stability to a volatile country.
 After election on October 20, 2019, the electoral authorities once again declared Morales the winner with 47 percent of ballots. But the result was disputed, leading to weeks of protests in

In [82]:
photo_filter_by_event(['afpevent:FAO84'])

,caption,event
0,"TOPSHOT - Handout picture released by the Mexican Foreign Ministry press office showing the Bolivian ex-President Evo Morales leaving Bolivia in an aircraft of the Mexican Air Force bound for Mexico on November 11, 2019. The Mexican government said Monday that it granted political asylum to Evo Morales, who resigned from the presidency of Bolivia on November 10, considering that ""his life and integrity are in danger,"" said Mexican Foreign Minister Marcelo Ebrard. - RESTRICTED TO EDITORIAL USE - MANDATORY CREDIT ""AFP PHOTO / MEXICO'S FOREIGN MINISTRY / HO"" - NO MARKETING - NO ADVERTISING CAMPAIGNS - DISTRIBUTED AS A SERVICE TO CLIENTS\r\n / AFP / Mexican Foreign Ministry / HO / RESTRICTED TO EDITORIAL USE - MANDATORY CREDIT ""AFP PHOTO / MEXICO'S FOREIGN MINISTRY / HO"" - NO MARKETING - NO ADVERTISING CAMPAIGNS - DISTRIBUTED AS A SERVICE TO CLIENTS",[afpevent:FAO84]
1,"TOPSHOT - Grab taken from a handout video released by Bolivia TV showing Bolivian President Evo Morales announcing his resignation on November 10, 2019 in a televised address from Cochabamba, Bolivia, caving in following three weeks of sometimes-violent protests over his disputed re-election after the army and police withdrew their backing. The Organization of American States carried out an audit of the election and on Sunday reported irregularities in just about every aspect that it examined: the technology used, the chain of custody of ballots, the integrity of the count, and statistical projections. - RESTRICTED TO EDITORIAL USE - MANDATORY CREDIT ""AFP PHOTO / BOLIVIA TV / HO"" - NO MARKETING - NO ADVERTISING CAMPAIGNS - DISTRIBUTED AS A SERVICE TO CLIENTS\r\n / AFP / Bolivia TV / HO / RESTRICTED TO EDITORIAL USE - MANDATORY CREDIT ""AFP PHOTO / BOLIVIA TV / HO"" - NO MARKETING - NO ADVERTISING CAMPAIGNS - DISTRIBUTED AS A SERVICE TO CLIENTS","[afpevent:FAN57, afpevent:FAO37, afpevent:FAO34, afpevent:FAO84]"
